In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Data reading

In [ ]:
Train = pd.read_csv("../input/bike-sharing-demand/train.csv")
Test = pd.read_csv("../input/bike-sharing-demand/test.csv")

## Data exploration

In [ ]:
Train.head()

In [ ]:
Test.head()

In [ ]:
Train.describe().T

In [ ]:
Test.describe().T

In [ ]:
Train.info()

In [ ]:
Test.info()

In [ ]:
# Correlation between the numerical columns
Train.loc[:, Train.dtypes != np.object].corr()

## EDA

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(30, 30))

sns.countplot(data = Train, x = 'season', ax = ax[0,0]);
ax[0,0].set_ylabel("Count", size = 30)
ax[0,0].set_xlabel("season", size = 30)



sns.countplot(data = Train, x = 'holiday', ax = ax[0,1]);
ax[0,1].set_ylabel("Count", size = 30)
ax[0,1].set_xlabel("holiday", size = 30)

sns.countplot(data = Train, x = 'workingday', ax = ax[1,0]);
ax[1, 0].set_ylabel("Count", size = 30)
ax[1, 0].set_xlabel("workingday", size = 30)

# plt.xticks(rotation = 90, size = 25);
# plt.yticks(size = 20);
# plt.tick_params(axis='both', which='major', fontsize=50)
# plt.xticks(fontsize=100)
# ax[0,0].set_xticklabels(fontsize=30)
# plt.yticks(fontsize=50)
# plt.xticks(fontsize=50)

# ax.tick_params(direction='out', length=6, width=2, colors='r',
#                grid_color='r', grid_alpha=0.5)
plt.show()

In [ ]:
Columns = ['weather', 'temp','atemp', 'humidity', 'windspeed']
sns.pairplot(Train, vars = Columns, diag_kind = 'kde')

## Data preparation

### Feature engineering

In [ ]:
Train['datetime'] = pd.to_datetime(Train['datetime'])

Train["year"] = Train["datetime"].dt.year
Train["hour"] = Train["datetime"].dt.hour
Train["dayofweek"] = Train["datetime"].dt.dayofweek

In [ ]:
Test['datetime'] = pd.to_datetime(Test['datetime'])

Test["year"] = Test["datetime"].dt.year
Test["hour"] = Test["datetime"].dt.hour
Test["dayofweek"] = Test["datetime"].dt.dayofweek


In [ ]:
# Train['ActiveHours']= Train['hour'].isin([8,17,18,19,20,21])
# Train['InactiveHour']= Train['hour'].isin([0,1,2,3,4])

In [ ]:
# Test['ActiveHours']= Test['hour'].isin([8,17,18,19,20,21])
# Test['InactiveHour']= Test['hour'].isin([0,1,2,3,4])

In [ ]:
# Train.drop(["datetime", "windspeed", "casual", "registered"], axis = 1, inplace=True)
# Test.drop(["datetime", "windspeed", ], axis = 1, inplace=True)

In [ ]:
Train.drop(["datetime", "casual", "registered"], axis = 1, inplace=True)
Test.drop(["datetime"], axis = 1, inplace=True)

In [ ]:
Train.isnull().sum()

In [ ]:
Train.duplicated().sum()

In [ ]:
Train.drop_duplicates(inplace = True)

In [ ]:
Test.isnull().sum()

In [ ]:
Test.duplicated().sum()

In [ ]:
# Test.drop_duplicates(inplace = True)

In [ ]:
sns.relplot(data = Train, x = 'hour', y = 'count', kind = 'line')

In [ ]:
sns.catplot(data = Train, x = 'hour', y = 'count', kind = 'bar')
plt.show()

In [ ]:
sns.boxplot(data = Train, x = 'hour', y = 'count')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (20, 15))
sns.boxplot(data = Train, x = 'temp', y = 'count')

plt.xticks(rotation = 90, size = 25);
plt.yticks(size = 20);
plt.ylabel("Count", size = 25)
plt.xlabel("temp", size = 25)
plt.show()

## Model

In [ ]:
Y = Train['count']
X = Train.copy()
X.drop('count', axis = 1, inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split
X_Train, X_Valid, Y_Train, Y_Valid = train_test_split(X, Y, train_size=0.9, test_size=0.1, random_state=0)

In [ ]:
import lightgbm as lgb
hyperparameters = { 'colsample_bytree': 0.725,  'learning_rate': 0.013,
                    'num_leaves': 56, 'reg_alpha': 0.754, 'reg_lambda': 0.071, 
                    'subsample': 0.523, 'n_estimators': 1093}

model = lgb.LGBMRegressor(**hyperparameters)
model.fit(X_Train, Y_Train)
preds1 = model.predict(X_Valid)

hyperparameters = { 'colsample_bytree': 0.639,  'learning_rate': 0.011,
                    'num_leaves': 30, 'reg_alpha': 0.351, 'reg_lambda': 0.587,
                   'subsample': 0.916, 'n_estimators': 2166}
model = lgb.LGBMRegressor(**hyperparameters, )
model.fit(X_Train, Y_Train)
preds2 = model.predict(X_Valid)


In [ ]:
from sklearn import metrics
# print('Coefficients: \n', model.coef_)
print("MAE: ", metrics.mean_absolute_error(Y_Valid, preds1))
# print("MAPE: ", metrics.mean_absolute_percentage_error(Y_Valid, ExtraTreesPrediction))



# print("MSLE: %.2f" % metrics.mean_squared_log_error(Y_Valid, preds1))
# print("RMSE:", np.sqrt(metrics.mean_squared_error(y_valid, preds_valid)))

print('Variance score (r2): %.2f' % metrics.r2_score(Y_Valid, preds1))
# print("r2 score:", metrics.r2_score(y_valid, preds_valid))

In [ ]:
final_predictions = model.predict(Test)

In [ ]:
Submission = pd.read_csv("../input/bike-sharing-demand/sampleSubmission.csv")
Submission.head()

In [ ]:
Submission['count'] = np.abs(final_predictions)

In [ ]:
Submission.to_csv("sub5.csv", index = False)